# __AutoML을 사용하여 예측 모델 만들기__

- 튜토리얼 난이도: ★☆☆☆☆
- 읽는 시간: 5분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_ml/regression/automl_regression.ipynb
- 참고 문서: [(캐글) Bike Sharing Demand](https://www.kaggle.com/competitions/bike-sharing-demand/overview)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">회귀 작업 이해하기</h4>
    <p>회귀 작업은 목푯값(Target)이 연속성을 지닌 숫자를 예측하기 위해 사용하는 <a href="https://ko.wikipedia.org/wiki/%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5">머신러닝(기계학습/Machine Learning)</a>의 한 형태입니다. 예를 들어, 기상 데이터가 주어졌을 때 내일 기온을 예측하거나, 특정지역의 집값을 예측할 때 사용할 수 있습니다.</p>
</div>

기업이 특정 금액을 광고에 사용할 경우 과거의 유사한 사례의 판매 성과 데이터를 활용하여 광고의 성과를 예측할 수 있습니다. 광고하고자 하는 제품에 대한 특성부터 제품을 판매하는 시기, 주변 시장 정보, 경쟁사의 판매량 정보, 대상 고객군에 대한 정의, 산업군의 시장 트렌드 등 데이터화 할 수 있는 모든 <a href="https://ko.wikipedia.org/wiki/%ED%8A%B9%EC%A7%95_(%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5)">특성(Feature)</a>이 입력자료가 될 수 있습니다. 입력 데이터에서 통제 가능한 정보를 바꿔가면서 최적의 판매성과를 예측해 볼 수 있고 예측성과에 따라 광고에 소비할 비용을 조정할 수도 있습니다. 이러한 회귀 모델을 이용해서 광고 성과를 향상시키고 판매량을 지속적으로 늘릴 수 있습니다.

__아래는 ThanoSQL 회귀 모델의 활용 및 예시입니다.__

 - 주식의 시가, 종가, 고가, 저가, 관련주 주가, 종합주가지수, 관련뉴스 등을 활용한 주식가격 예측 (금융)
 - 기기/설비의 온도, 진동, 소리 등 센서데이터를 이용한 고장 확률 및 잔존 수명 예측 (제조)
 - 날씨, 기온, 운량, 일사량 등을 활용한 태양광 에너지 발전량 예측 (에너지)
 - 과거 수요량 트렌드, 유가 및 환율 변동 등을 활용한 수요예측 (원자재)

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 대표적인 머신러닝 경진대회 플랫폼인 <a href="https://www.kaggle.com/">캐글</a>의 입문자를 위한 Bike Sharing Demand 데이터 세트를 사용하여 자전거 수요 예측 회귀 모델을 만듭니다. 이 대회의 목표는 아래와 같습니다. (참고로, 해당 대회의 데이터는 2011년부터 2012년까지 날짜와 시간, 기온, 습도, 풍속 등의 정보를 기반으로 자전거 대여 횟수가 기재되어 있습니다.)</p>
</div>

__특정 날짜에서 시간대별 자전거 대여 수량 예측하기__

ThanoSQL에서는 자동화된 머신러닝(__AutoML__)을 제공합니다. 본 튜토리얼에서는 <strong>AutoML</strong>을 사용하여 자전거 대여 수량을 예측합니다. ThanoSQL에서 제공하는 <strong>AutoML</strong>은 모델(Model)개발을 위한 프로세스를 자동화하고, 데이터 과학(Data Science)에 대한 전문지식이 없어도 데이터의 수집 및 저장, 머신러닝 모델의 개발 및 배포(엔드투엔드 머신러닝 파이프라인)를 하나의 언어(ThanoSQL)만으로도 가능하도록 지원합니다.

__ThanoSQL의 자동화 된 머신러닝을 사용하면 다음과 같은 장점이 있습니다.__

1. 광범위한 프로그래밍 또는 데이터과학에 대한 지식이 없어도 머신러닝 솔루션의 구현 및 배포가 가능
2. 개발모델의 배포에 들어가는 시간 및 리소스를 절약
3. 의사결정을 위해 보유하고 있는 데이터를 이용한 신속한 문제해결이 가능

이제부터 ThanoSQL을 사용하여 간단하게 자전거 대여 수량을 예측하는 회귀 모델을 만들어 봅니다.

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET bike_sharing_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY bike_sharing_train
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/bike_sharing_data/bike_sharing_train.csv'

Success


In [4]:
%%thanosql
COPY bike_sharing_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/bike_sharing_data/bike_sharing_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

회귀 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __bike_sharing_train__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM bike_sharing_train
LIMIT 5

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00,1,0,0,1,9.84,14.395,75,0.0,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>bike_sharing_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다. 2011년 1월부터 2012년 12월까지 날짜와 시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수에 대한 정보를 담고 있습니다.</p>
    <ul>
        <li>datetime: 시간별 날짜</li>
        <li>season: 계절(1 = 봄, 2 = 여름, 3 = 가을, 4 = 겨울)</li>
        <li>holiday: 휴일(0 = 휴일이 아닌 날, 1 = 주말을 제외한 국경일 등의 휴일)</li>
        <li>workingday: 작업일(0 = 주말 및 휴일, 1 = 주말 및 휴일이 아닌 주중)</li>
        <li>weather: 날씨</li>
        <li>temp: 온도</li>
        <li>atemp: 체감온도</li>
        <li>humidity: 상대습도</li>
        <li>windspeed: 풍속</li>
        <li>count: 대여 횟수</li>
    </ul>
</div>

## __2. 회귀 모델 생성__

이전 단계에서 확인한 __bike_sharing_train__ 테이블을 사용하여 자전거 수요 예측 회귀 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>bike_regression</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 8 min)

In [6]:
%%thanosql
BUILD MODEL bike_regression
USING AutomlRegressor
OPTIONS (
    target_col='count',
    impute_type='simple',
    datetime_attribs=['datetime'],
    time_left_for_this_task=300,
    overwrite=True
    )
AS
SELECT *
FROM bike_sharing_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>bike_regression</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>AutomlRegressor</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"target_col": 회귀 예측 모델의 목푯값이 담겨 있는 컬럼명 (str, default: 'target')</li>
            <li>"impute_type": 데이터 테이블의 빈 값(NaN)을 처리하는 방법 설정 (str, optional, 'simple'|'iterative', default: 'simple')</li>
            <li>"datetime_attribs": 날짜 형식의 데이터가 담겨 있는 컬럼명 리스트 (list[str], optional)</li>
            <li>"time_left_for_this_task": 적합한 회귀 예측 모델을 찾는데 소요되는 초 단위 시간 (int, optional, default: 60)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 생성된 모델 평가__

아래의 쿼리문을 실행하여 이전 단계에서 만든 예측 모델의 성능을 평가합니다.

In [7]:
%%thanosql
EVALUATE USING bike_regression
OPTIONS (
    target_col='count'
    )
AS
SELECT *
FROM bike_sharing_train

,metric,score
0,MAE,78.6563
1,MSE,10986.4542
2,R2,0.2292
3,RMSLE,1.3861
4,MAPE,0.5028


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>EVALUATE USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>bike_regression</strong> 모델을 평가합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 평가에 사용할 옵션을 지정합니다.
        <ul>
            <li>"target_col": 회귀 예측 모델의 목푯값이 담겨있는 컬럼명 (str, default: 'target')</li>
        </ul>
        </li>
    </ul>
</div>

<div class="admonition warning">
    <h4 class="admonition-title">평가용 데이터 세트</h4>
    <p>평가용 데이터 세트는 학습 데이터 세트의 일부를 분리하여 학습에 사용되지 않아야 하나 튜토리얼에서는 편의상 학습 데이터를 사용합니다.</p>
</div>

## __4. 생성된 모델을 사용하여 자전거 대여 수량 예측__

이전 단계에서 생성한 수요 예측 모델을 사용해 10개의 데이터에 대한 자전거 대여 수량을 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, <strong>bike_sharing_test</strong>)를 사용합니다.

In [8]:
%%thanosql
PREDICT USING bike_regression
OPTIONS (
    result_col='predict_result'
    )
AS
SELECT *
FROM bike_sharing_test
LIMIT 10

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,predict_result
0,2011-01-20 0:00,1,0,1,1,10.66,11.365,56,26.0027,102.836334
1,2011-01-20 1:00,1,0,1,1,10.66,13.635,56,0.0000,92.060480
2,2011-01-20 2:00,1,0,1,1,10.66,13.635,56,0.0000,92.060480
3,2011-01-20 3:00,1,0,1,1,10.66,12.880,56,11.0014,95.181085
4,2011-01-20 4:00,1,0,1,1,10.66,12.880,56,11.0014,95.181085
5,2011-01-20 5:00,1,0,1,1,9.84,11.365,60,15.0013,91.816701
6,2011-01-20 6:00,1,0,1,1,9.02,10.605,60,15.0013,87.213365
7,2011-01-20 7:00,1,0,1,1,9.02,10.605,55,15.0013,87.054590
8,2011-01-20 8:00,1,0,1,1,9.02,10.605,55,19.0012,88.568595
9,2011-01-20 9:00,1,0,1,2,9.84,11.365,52,15.0013,103.445460


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>bike_regression</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>"result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __5. 튜토리얼을 마치며__

이번 튜토리얼에서는 [캐글](https://www.kaggle.com)의 Bike Sharing Demand 데이터 세트를 사용하여 자전거 수요 예측 회귀 모델을 만들어 보았습니다. 초급 단계 튜토리얼인만큼 정확도 향상을 위한 과정보다는 전반적인 프로세스 위주의 설명으로 진행 하였습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>회귀 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>